# Parse RSS Feeds and Write a Hugo markdown post automatically.

This script was written to automatically parse my blog feed and write it into a YAML based markdown file so I could run a Hugo based web site easier. The only requirementa are to use Feedparser and Pandas which you can install by doing 'pip install feedparser' and 'pip install pandas'.

In [ ]:
'''Uncomment this line if you want to install feedparser and pandas'''
#!pip install feedparser
#!pip install pandas

In [ ]:
#!/usr/bin/env python
import sys
import os
import random

import feedparser

import pandas as pd
import csv

from urllib.parse import urlparse

In [ ]:
#Set the markdown post write directory
mydir = '/Users/thomasott/Dropbox/hugo/deoxy.xyz/content/items/' #put the directory where you want to save the YAML based .md files

In [ ]:
#Parse the Feed
myfeed = feedparser.parse('https://neuralmarkettrends.com/index.xml') #Put your own feed here. 

# Check the parsed feed
The following are just some internal checks to make sure the feed was parsed correctly. 
Some feeds may use 'date' vs 'publised' or other things that could cause this scrip to break

In [ ]:
myfeed

In [ ]:
myfeed.entries[10].title

In [ ]:
myfeed.entries[1]

In [ ]:
myfeed.entries[10].published

In [ ]:
myfeed.entries[10].link

In [ ]:
o = urlparse(myfeed.entries[10].link).hostname

In [ ]:
o

In [ ]:
len (myfeed)

In [ ]:
myfeed.entries[10].summary

In [ ]:
output = []
entry = len(myfeed)

for i in range(entry):
    j = 0
    try:
        tmp = (myfeed.entries[i].title, myfeed.entries[i].published, myfeed.entries[i].link, urlparse(myfeed.entries[i].link).hostname, 
           myfeed.entries[i].tags[j].term)
    except:
        tmp = (myfeed.entries[i].title, myfeed.entries[i].published, myfeed.entries[i].link, urlparse(myfeed.entries[i].link).hostname, 
           'untagged')
    output.append(tmp)
    

In [ ]:
#output

In [ ]:
df = pd.DataFrame(output, columns=['Title', 'Date', 'ItemURL', 'Sites', 'Tags'])

In [ ]:
#Make your exclusions here
df = df[df.Sites != 'twitter.com']
df = df[df.Sites != 'medium.com']

In [ ]:
df

In [ ]:
#Use Regex to strip any tags if they're in the in the wrong format
# Added a REGEX fix for Titles that could cause Hugo not to parse the file. 

import re

for index, row in df.iterrows():
    row['Tags'] = re.sub("\s+", ",", row['Tags'].strip())
    row['Title'] = re.sub("/", "-", row['Title'].strip())

In [ ]:
df = df[df["Title"].apply(lambda x:x not in ['Untitled'])]

In [ ]:
df

In [ ]:
for index, row in df.iterrows():
    with open(mydir+row['Title']+'.md', 'w') as f:
        f.write('--- \ntitle: "'+row['Title']+'"\n'+
                'date: '+row['Date']+'\n'+
                'itemurl: "'+row['ItemURL']+'"\n'+
                'sites: "'+row['Sites']+'"\n'+
                'tags: ['+row['Tags']+']\n'+
                '---')
        
        #f.write('---\n', 'title:', row['Title'], '\n', 'date:', row['Date'], '\n', 'itemurl:', row['ItemURL'], '\n', 'sites:', row['Sites'], 'summary:', row['Summary'], '\n', '---'))
        #print('---\n', 'title:', row['Title'], '\n', 'date:', row['Date'], '\n', 'itemurl:', row['ItemURL'], '\n', 'sites:', row['Sites'], 'summary:', row['Summary'], '\n', '---')
